In [7]:
import retro
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

%matplotlib inline

# Basic Setup for Enviroment
env = retro.make('NbaJamTE-Snes')
obs = env.reset()

# Use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [ ]:
print(env.observation_space)
img = plt.imshow(env.render(mode='rgb_array')) # only call this once